# XML, HTML, and Web Scraping

JSON and XML are two different ways to represent hierarchical data. Which one is better? There are lots of articles online which discuss similarities and differences between JSON and XML and their advantages and disadvantages. Both formats are still in current usage, so it is good to be familiar with both. However, JSON is more common, so we'll focus on working with JSON representations of hierarchical data.

The reading covered an example of using Beautiful Soup to parse XML. Rather than doing another example XML now, we'll skip straight to scraping HTML from a webpage. Both HTML and XML can be parsed in a similar way with Beautiful Soup.

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

## Scraping an HTML table with Beautiful Soup

Open the URL https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population and scroll down until you see a table of the cities in the U.S. with population over 100,000 (as of Jul 1, 2022). We'll use Beautiful Soup to scrape information from this table.

Read in the HTML from the ULR using the `requests` library.

In [2]:
# YOUR CODE HERE
URL = "https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population"
HEADERS = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

response = requests.get(URL, headers=HEADERS)

Use Beautiful Soup to parse this string into a tree called `soup`

In [3]:
# YOUR CODE HERE
soup = BeautifulSoup(response.content, "html.parser")

To find an HTML tag corresponding to a specific element on a webpage, right-click on it and choose "Inspect element". Go to the cities table Wikipedia page and do this now.

You should find that the cities table on the Wikipedia page corresponds to the element

```
<table class="wikitable sortable jquery-tablesorter" style="text-align:center">
```

There are many `<table>` tags on the page.

In [4]:
len(soup.find_all("table"))

10

We can use attributes like `class=` and `style=` to narrow down the list.

In [5]:
len(soup.find_all("table",
                  attrs={
                      "class": "wikitable",
                      "style": "text-align:right"}
                  ))

3

At this point, you can manually inspect the tables on the webpage to find that the one we want is the first one (see `[0]` below). We'll store this as `table`.

In [6]:
table = soup.find_all("table",
                  attrs={
                      "class": "wikitable",
                      "style": "text-align:right"}
                  )[0]

**Now you will write code to scrape the information in `table` to create a Pandas data frame with one row for each city and columns for: city, state, population (2022 estimate), and 2020 land area (sq mi).** Refer to the Notes/suggestions below as you write your code. A few Hints are provided further down, but try coding first before looking at the hints.

Notes/suggestions:

- Use as a guide the code from the reading that produced the data frame of Statistics faculty
- Inspect the page source as you write your code
- You will need to write a loop to get the information for all cities, but you might want to try just scraping the info for New York first
- You will need to pull the text from the tag. If `.text` returns text with "\n" at the end, try `.get_text(strip = True)` instead of `.text`
- Don't forget to convert to a Pandas Data Frame; it should have 333 rows and 4 columns
- The goal of this exercise is just to create the Data Frame. If you were going to use it --- e.g., what is the population density for all cities in CA? --- then you would need to clean the data first (to clean strings and convert to quantitative). (You can use Beautiful Soup to do some of the cleaning for you, but that goes beyond our scope.)

In [7]:
# YOUR CODE HERE. ADD AS MANY CELLS AS NEEDED
# initialize an empty list

rows = []

for location in table.find_all("tr")[2:]:
    cells = location.find_all(["td"])
    
    #finding the city name in the given city column and storing it under city
    city_tag = cells[0].find("title") or cells[0]
    city = city_tag.get_text(strip=True)
    
    #adding a state column using the data
    state_tag = cells[1].find("a") or cells[1]
    state = state_tag.get_text(strip=True)

    #adding 2024 population estimate into rows 
    pop_tag = cells[2].find("") or cells[2]
    pop = pop_tag.get_text(strip=True)

    #adding 2020 land area into our rows 
    area_tag = cells[5].find("") or cells[5]
    area = area_tag.get_text(strip=True)

    rows.append({
        "City": city,
        "State": state,
        "2024 Population": pop,
        "2020 Land Area": area
    })


In [8]:
rows

[{'City': 'New York[c]',
  'State': 'NY',
  '2024 Population': '8,478,072',
  '2020 Land Area': '300.5'},
 {'City': 'Los Angeles',
  'State': 'CA',
  '2024 Population': '3,878,704',
  '2020 Land Area': '469.5'},
 {'City': 'Chicago',
  'State': 'IL',
  '2024 Population': '2,721,308',
  '2020 Land Area': '227.7'},
 {'City': 'Houston',
  'State': 'TX',
  '2024 Population': '2,390,125',
  '2020 Land Area': '640.4'},
 {'City': 'Phoenix',
  'State': 'AZ',
  '2024 Population': '1,673,164',
  '2020 Land Area': '518.0'},
 {'City': 'Philadelphia[d]',
  'State': 'PA',
  '2024 Population': '1,573,916',
  '2020 Land Area': '134.4'},
 {'City': 'San Antonio',
  'State': 'TX',
  '2024 Population': '1,526,656',
  '2020 Land Area': '498.8'},
 {'City': 'San Diego',
  'State': 'CA',
  '2024 Population': '1,404,452',
  '2020 Land Area': '325.9'},
 {'City': 'Dallas',
  'State': 'TX',
  '2024 Population': '1,326,087',
  '2020 Land Area': '339.6'},
 {'City': 'Jacksonville[e]',
  'State': 'FL',
  '2024 Populat

Hints:

- Each city is a row in the table; find all the `<tr>` tags to find all the cities
- Look for the `<td>` tag to see table entries within a row
- The rank column is represented by `<th>` tags, rather than `<td>` tags. So within a row, the first (that is, `[0]`) `<td>` tag corresponds to the city name.

## Aside: Scraping an HTML table with Pandas



The Pandas command `read_html` can be used to scrape information from an HTML table on a webpage.

We can call `read_html` on the URL.

In [45]:
pd.read_html("https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population", storage_options={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"})

[                    Population tables of U.S. cities
 0  The skyline of New York City, the most populou...
 1                                             Cities
 2  PopulationAreaDensityEthnic identityForeign-bo...
 3                                        Urban areas
 4             Populous cities and metropolitan areas
 5                                 Metropolitan areas
 6  184 combined statistical areas935 core-based s...
 7                                        Megaregions
 8  Related population listsNorth American metro a...
 9                                                vte,
     0                               1
 0 NaN                   State capital
 1 NaN            State's largest city
 2 NaN  State capital and largest city
 3 NaN                 Federal capital,
             City  ST 2024 estimate 2020 census  ... 2020 land area 2020 density                                      Location
             City  ST 2024 estimate 2020 census  ...            km2        / mi2  

However, this scrapes all the tables on the webpage, not just the one we want. As with Beautiful Soup, we can narrow the search by specifying the table attributes.

In [48]:
pd.read_html(URL, storage_options={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"})


[                    Population tables of U.S. cities
 0  The skyline of New York City, the most populou...
 1                                             Cities
 2  PopulationAreaDensityEthnic identityForeign-bo...
 3                                        Urban areas
 4             Populous cities and metropolitan areas
 5                                 Metropolitan areas
 6  184 combined statistical areas935 core-based s...
 7                                        Megaregions
 8  Related population listsNorth American metro a...
 9                                                vte,
     0                               1
 0 NaN                   State capital
 1 NaN            State's largest city
 2 NaN  State capital and largest city
 3 NaN                 Federal capital,
             City  ST 2024 estimate 2020 census  ... 2020 land area 2020 density                                      Location
             City  ST 2024 estimate 2020 census  ...            km2        / mi2  

This still returns 3 tables. As we remarked above, the table that we want is the first one (see `[0]` below).

In [59]:
df_cities2 = pd.read_html(URL, storage_options={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"})[2]
df_cities2

City  ST 2024 estimate 2020 census  ... 2020 land area 2020 density                                      Location
            City  ST 2024 estimate 2020 census  ...            km2        / mi2  / km2                               Location
0    New York[c]  NY       8478072     8804190  ...          778.3        29298  11312    40°40′N 73°56′W﻿ / ﻿40.66°N 73.94°W
1    Los Angeles  CA       3878704     3898747  ...         1216.0         8304   3206  34°01′N 118°25′W﻿ / ﻿34.02°N 118.41°W
2        Chicago  IL       2721308     2746388  ...          589.7        12061   4657    41°50′N 87°41′W﻿ / ﻿41.84°N 87.68°W
3        Houston  TX       2390125     2304580  ...         1658.6         3599   1390    29°47′N 95°23′W﻿ / ﻿29.79°N 95.39°W
4        Phoenix  AZ       1673164     1608139  ...         1341.6         3105   1199  33°34′N 112°05′W﻿ / ﻿33.57°N 112.09°W
..           ...  ..           ...         ...  ...            ...          ...    ...                                    ...
341      Deltona  FL        100513       93692  ...           96.6         2512    970    28°55′N 81°13′W﻿ / ﻿28.91°N 81.21°W
342  Federal Way  WA        100252      101030  ...           57.8         4530   1750  47°19′N 122°20′W﻿ / ﻿47.31°N 122.34°W
343   San Angelo  TX        100159       99893  ...          154.6         1673    646  31°26′N 100°27′W﻿ / ﻿31.44°N 100.45°W
344        Tracy  CA        100136       93000  ...           67.1         3591   1386  37°44′N 121°27′W﻿ / ﻿37.73°N 121.45°W
345      Sunrise  FL        100128       97335  ...           42.0         6008   2320    26°10′N 80°16′W﻿ / ﻿26.17°N 80.26°W

[346 rows x 10 columns]

Wait, that seemed much easier than using Beautiful Soup, and it returned a data frame, and we even got for free some formatting like removing the commas from the population! Why didn't we just use `read_html` in the first place? It's true the `read_html` works well when scraping information from an HTML *table*. Unfortunately, you often want to scrape information from a webpage that isn't conveniently stored in an HTML table, in which case `read_html` won't work. (It only searches for `<table>`, `<th>`, `<tr>`, and `<td>` tags, but there are many other HTML tags.) Though Beautiful Soup is not as simple as `read_html`, it is more flexible and thus more widely applicable.

## Scraping information that is NOT in a `<table>` with Beautiful Soup

The Cal Poly course catalog http://catalog.calpoly.edu/collegesandprograms/collegeofsciencemathematics/statistics/#courseinventory contains a list of courses offered by the Statistics department. **You will scrape this website to obtain a Pandas data frame with one row for each DATA or STAT course and two columns: course name and number (e.g, DATA 301. Introduction to Data Science) and term typically offered (e.g., Term Typically Offered: F, W, SP).**

Note: Pandas `read_html` is not help here since the courses are not stored in a `<table>.`

In [60]:
pd.read_html("http://catalog.calpoly.edu/collegesandprograms/collegeofsciencemathematics/statistics/#courseinventory")

[                                        Program name   Program type
 0                              Actuarial Preparation          Minor
 1  Cross Disciplinary Studies Minor in Bioinforma...          Minor
 2   Cross Disciplinary Studies Minor in Data Science          Minor
 3                                         Statistics  BS, MS, Minor]


Notes/suggestions:


- Inspect the page source as you write your code
- The courses are not stored in a `<table>`. How are they stored?
- You will need to write a loop to get the information for all courses, but you might want to try just scraping the info for DATA 100 first
- What kind of tag is the course name stored in? What is the `class` of the tag?
- What kind of tag is the quarter(s) the course is offered stored in? What is the `class` of the tag? Is this the only tag of this type with the class? How will you get the one you want?
- You don't have to remove the number of units (e.g., 4 units) from the course name and number, but you can try it if you want
- You will need to pull the text from the tag. If `.text` returns text with "\n" at the end, try `get_text(strip = True)` instead of `text`
- Don't forget to convert to a Pandas Data Frame; it should have 74 rows and 2 columns
- The goal of this exercise is just to create the Data Frame. If you were going to use it then you might need to clean the data first. (You can use Beautiful Soup to do some of the cleaning for you, but that goes beyond our scope.)



In [97]:
response2 = requests.get("https://catalog.calpoly.edu/collegesandprograms/collegeofsciencemathematics/statistics/#courseinventory")
web =BeautifulSoup(response2.content, "html.parser")

cal = web.find_all(
                  attrs={
                      "id": "courseinventorycontainer"})[0]

In [144]:
# YOUR CODE HERE. ADD AS MANY CELLS AS NEEDED
rows2 = []

for course in cal.find_all("div", attrs = {"class": "courseblock"})[0:]:

    name_tag = course.find("strong")    
    name = (name_tag.get_text(strip = True))[0:4]
    num = (name_tag.get_text(strip = True))[5:8]

    name_long_tag = course.find("strong")
    name_long = (name_long_tag.get_text(strip=True)).split('.',2)[1]
    
    term_tag = course.find("p", attrs = {"class": "noindent"})
    term = term_tag.get_text()[24:]
 


    rows2.append({
        "title": name + " " + num,
        "Course Name": name_long,
        "term" : term 
    })


In [145]:
rows2

[{'title': 'DATA 100',
  'Course Name': ' Data Science for All I',
  'term': 'F, W, SP'},
 {'title': 'DATA 301',
  'Course Name': ' Introduction to Data Science',
  'term': 'F, W, SP'},
 {'title': 'DATA 401',
  'Course Name': ' Data Science Process and Ethics',
  'term': 'F'},
 {'title': 'DATA 402',
  'Course Name': ' Mathematical Foundations of Data Science',
  'term': 'F'},
 {'title': 'DATA 403',
  'Course Name': ' Data Science Projects Laboratory',
  'term': 'F'},
 {'title': 'DATA 441',
  'Course Name': ' Bioinformatics Capstone I',
  'term': 'W'},
 {'title': 'DATA 442',
  'Course Name': ' Bioinformatics Capstone II',
  'term': 'SP'},
 {'title': 'DATA 451', 'Course Name': ' Data Science Capstone I', 'term': 'W'},
 {'title': 'DATA 452',
  'Course Name': ' Data Science Capstone II',
  'term': 'SP'},
 {'title': 'DATA 472',
  'Course Name': ' Data Science Seminar',
  'term': 'F, W, SP'},
 {'title': 'STAT 130',
  'Course Name': ' Introduction to Statistical Reasoning',
  'term': 'F, W, S

Hints:

- Each course is represented by a `<div>` with `class=courseblock`, so you can find all the courses with `soup.find_all("div", {"class": "courseblock"})`
- The course name is in a `<p>` tag with `class=courseblocktitle`, inside a `<strong>` tag. (Though I don't think we need to find the strong tag here.)
- The term typically offered is in `<p>` tag with `class=noindent`. However, there are several tags with this class; term typically offered is the first one.
- If you want to use Beautiful Soup to remove the course units (e.g., 4 units), find the `<span>` tag within the course name tag and `.extract()` this span tag